In [2]:
!pip install -U sentence_transformers

You should consider upgrading via the 'C:\Users\hp\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install -U faiss-cpu

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hp\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import faiss

c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [3]:
import csv

In [6]:
df = pd.read_csv('synthetic_names_samples.csv')

In [5]:
doc_embeddings = model.encode(df['transcription'].tolist())
doc_embeddings = np.array(doc_embeddings).astype('float32') # doing this for faiss

In [22]:
dimension = doc_embeddings.shape[1] #384 in this case.
index = faiss.IndexFlatIP(dimension)

In [23]:
index.add(doc_embeddings)

In [24]:
question = "What happened to the patient called Michael Dunstan who takes hydrochlorothiazide for hypertension?"
question_embedding = model.encode([question]).astype('float32')

In [25]:
k = 10 # number of documents.
distances, indices = index.search(question_embedding, k)

In [26]:
for i, idx in enumerate(indices[0]):
    doc_index = df.iloc[idx]['index']
    doc_text = df.iloc[idx]['transcription']
    print(f"Index: {doc_index}, Distance: {distances[0][i]:.4f}, Document: {doc_text}")

Index: 13, Distance: 0.7119, Document: NAME : Michael Dunstan  PAST MEDICAL HISTORY:,  Significant for hypertension.  The patient takes hydrochlorothiazide for this.  She also suffers from high cholesterol and takes Crestor.  She also has dry eyes and uses Restasis for this.  She denies liver disease, kidney disease, cirrhosis, hepatitis, diabetes mellitus, thyroid disease, bleeding disorders, prior DVT, HIV and gout.  She also denies cardiac disease and prior history of cancer.,PAST SURGICAL HISTORY: , Significant for tubal ligation in 1993.  She had a hysterectomy done in 2000 and a gallbladder resection done in 2002.,MEDICATIONS: , Crestor 20 mg p.o. daily, hydrochlorothiazide 20 mg p.o. daily, Veramist spray 27.5 mcg daily, Restasis twice a day and ibuprofen two to three times a day.,ALLERGIES TO MEDICATIONS: , Bactrim which causes a rash.  The patient denies latex allergy.,SOCIAL HISTORY: , The patient is a life long nonsmoker.  She only drinks socially one to two drinks a month. 

In [2]:
from joblib import load, dump

In [28]:
dump(model, "sbert.joblib")
dump(index, "faiss.joblib")

['faiss.joblib']

In [7]:
docs = df['transcription'].to_list()

In [8]:
dump(docs, "docs.joblib")

['docs.joblib']